In [ ]:
import re
import json
import numpy as np
import pandas as pd
from copy import deepcopy
#Account Age
import time
from datetime import datetime

#SQL
import mysql.connector 
from mysql.connector import errorcode
import config
import json

import matplotlib.pyplot as plt

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

## Product Analysis 

1. look into trending products
2. Find average price of trending products 

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['Successful', 'sale', 'to', 'Sold', 'sold', 'an', 'Bought', 'a', 'from', 'purchased', 'some', 'cash', 'for', 'with', 'was', 'the', 'bought', 'it', 'is', 'Purchased', 'trade', 'Traded', 'great', 'new', 
 'shipping', 'received', 'my', 'of', 'supply', 'in', 'me', 'Confirmed', 'very', 'off', 'Great', 'broken', 'shipped',  
 'traded', 'as', 'I', 'set', 'parts', 'on', 'locally', 'you', 'but', 'Thanks', 'Thanks!', 'Thank', 'used', ']', '[', '(', ')', ' Local Cash', ' PayPal', '  Paypal', ' Local', ' local cash', '  PayPal', ' Paypal', ' Cash', ' Local cash', ' cash', ' paypal', '  Local Cash', 'Trades', 'PayPal', 'Local', 'Cash', 'Paypal', 'local', ' paypal', 'Verified', ' Steam Link'] )

In [ ]:
def text_process(mess):
    lemmatizer = WordNetLemmatizer()
    temp_list = []
    for product in title:
        product = product.split()
        if word not in stop_words:
            word = lemmatizer.lemmatize(word)
            temp_list.append(word)
            
    return temp_list

In [ ]:
def sql_to_df():
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.passwd,
        database = config.db_name
    )
    cur = cnx.cursor()
    cur.execute("""SELECT * FROM Submissions.submissions;""")
    df_sql = pd.DataFrame(cur.fetchall())
    df_sql.columns = [x[0] for x in cur.description]
    df_sql.drop('index', axis=1, inplace=True)
    cur.close()
    cnx.close()
    return df_sql

In [ ]:
df = sql_to_df()

In [ ]:
def clean_titles(df):
    titles = df['title'].values.tolist()
    
    separate_titles = []
    product_list = []
    final_product_list = []
    temp_string = ''
    for title in titles:
        new_string = re.sub(r'\[.*?\]', '', title)
        if ',' in new_string:
            separate_titles.append(new_string.split(','))
        elif '|' in new_string:
            new_string = new_string.replace('|', ',')
            separate_titles.append(new_string.split(','))
        else:
            separate_titles.append([new_string])
            
    
    for title in separate_titles:
        for product in title:
            product_list.append(product)
            

    lemmatizer = WordNetLemmatizer()
    for product in product_list: 
        product = product.split()
        for word in product:
            if word not in stop_words:
                word = lemmatizer.lemmatize(word)
                temp_string += ' ' + word.lower()
        if temp_string != '':
            final_product_list.append(temp_string[1:])
        temp_string = ''
        
    return final_product_list
    